# Lab 10
Understanding Docker Networks

What is Container Networking?
Container networking also called Software Defined Networking (SDN) is what allows Docker networkgin subsystems to work. 

Some defalt Docker Netwrok drivers are:
- Bridge
    - a Network that is User-defined to allow for all containers connected on the same network to communicate with eachother. 
- Host
    - a container is attahed to a Docker host. Any traffic routed to the container must be routed from the host system.
- Overlay
    - Created in a network spanning multiple docker hosts. Think each of your docker containers in the lab talking to eachother
- Macvlan
    - used in complicated networking to assing logical addresses based on MAC addresses to the virtual network interfaces making it apear as if the container has a physical network interaface connected to the network. Used in legacy applications.
- None
    -  the container only has a loopback interface so it can talk to itself.



# Part 1 - Setup

In [2]:
!docker run -d --name nginx --network=none -p 81:81 nginx

docker: Error response from daemon: Conflict. The container name "/nginx" is already in use by container "48de9566f04970864546f52fda2653a43e23ceb89a55764b5f9083ec4a83032d". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


## Note:
**If you get this error:**

*docker: permission denied while trying to connect to the Docker daemon ...*

run this in wsl: 
``` bash
sudo usermod -aG docker $USER
newgrp docker
sudo systemctl restart docker
```
Run it again....

you should get a hash number YAY!

`cbb2f01cd1b...`

--------------------------------------
If you are still getting the error


run this:
``` bash
sudo chmod 666 /usr/docker.sock
```
--------------------------------------
**If you get this error you did things right**

docker: Error response from daemon: Conflict. The container name "/nginx

--------------------------------------


## we now have an nginx container 

NGINX serves static content (like HTML, CSS, JavaScript, images) efficiently, improving website performance.

In [1]:
!curl localhost

#the curl command should fail on our system 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
curl: (7) Failed to connect to localhost port 80 after 2225 ms: Could not connect to server


Lets run it on the docker container 

In [4]:
!docker exec -it nginx2 /bin/sh -c "curl google.com"



Error response from daemon: No such container: nginx2


* curl:(6) Could not resolve host: google.com

As you can see the nginx container can not talk to google.com

# Part 2 - Understanding the network 
Docker comes with a subcommand for handling Docker networks.

In [5]:
!docker network


Usage:  docker network COMMAND

Manage networks

Commands:
  connect     Connect a container to a network
  create      Create a network
  disconnect  Disconnect a container from a network
  inspect     Display detailed information on one or more networks
  ls          List networks
  prune       Remove all unused networks
  rm          Remove one or more networks

Run 'docker network COMMAND --help' for more information on a command.


running `docker network ls` will show us all the names of the networks.


In [ ]:
!docker network ls 

NETWORK ID     NAME      DRIVER    SCOPE
d2b08c95b890   bridge    bridge    local
d873972cad3b   host      host      local
f7b0737ec118   none      null      local


In the above you can see that there is a none network. 

When we setup the docker nginx server we did not give it a network

lets look for the nginx server

In [23]:
!docker network inspect none 

[
    {
        "Name": "none",
        "Id": "f7b0737ec1188b800684be7edfaea4ba5d2871bc12f00933bf55712a110ee27d",
        "Created": "2025-04-18T12:24:26.478234728-07:00",
        "Scope": "local",
        "Driver": "null",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": null
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "cbb2f01cd1b1f17d502470e765159bc3400ca19f6862028bbc0ebb2c065f8183": {
                "Name": "nginx2",
                "EndpointID": "64ba2adbfd4b93f0be8a480f884b4417c5e69f2f221294786f127342b46cf6b8",
                "MacAddress": "",
                "IPv4Address": "",
                "IPv6Address": ""
            }
        },
        "Options": {},
        "Labels": {}
    }
]


## OK Enough of that nonsense
Lets install docker and use the API to do some work this is a python class after all

In [3]:
!pip install docker


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import docker

client = docker.from_env()

print("Docker version:", client.version()['Version'])


Docker version: 27.5.1


Lets create a quick code to list the containers using the docker client

In [5]:
def list_running_containers():
    containers = client.containers.list(all=True)# list all containers
    #containers = client.containers.list() #list only running containers
    for container in containers:
        print(f"{container.name} - {container.status}")

list_running_containers()

adminer - exited
mysql - exited
nginx - exited
epic_edison - exited
tender_bell - exited
elated_borg - exited


Here you can see the running container and no surprise it is the nginx we started before


In [9]:
#lest identify the network of the nginx container
def show_container_ip(container_name):
    container = client.containers.get(container_name)
    networks = container.attrs['NetworkSettings']['Networks']
    
    for net_name, net_data in networks.items():
        print(f"{container_name} in network '{net_name}' has IP: {net_data['IPAddress']}")


show_container_ip("nginx")


nginx in network 'none' has IP: 


again no surprise there is no IP address as it does not have a network. 

### ok I lied back to creating more stuff though the CLI

In [10]:
#setup another container for mysql
!docker run -d --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8
#Note the -d means detached mode so no container is using created just the MySQL database 

5e66c302f1d2f7e672327ffe8b914418b801b3e308ab4141e2aa0a44f8b61de3


Unable to find image 'mysql:8' locally
8: Pulling from library/mysql
cea172a6e83b: Pulling fs layer
6c228f7bea3b: Pulling fs layer
290ec4c227a4: Pulling fs layer
cb416885cc1f: Pulling fs layer
10eaa68c5921: Pulling fs layer
07e8f079852f: Pulling fs layer
d1e5287da18a: Pulling fs layer
aad4571eacc0: Pulling fs layer
182039f5bee3: Pulling fs layer
b3dd4e5cd318: Pulling fs layer
07e8f079852f: Waiting
d1e5287da18a: Waiting
aad4571eacc0: Waiting
b3dd4e5cd318: Waiting
182039f5bee3: Waiting
cb416885cc1f: Waiting
10eaa68c5921: Waiting
6c228f7bea3b: Verifying Checksum
6c228f7bea3b: Download complete
290ec4c227a4: Verifying Checksum
290ec4c227a4: Download complete
10eaa68c5921: Verifying Checksum
10eaa68c5921: Download complete
cb416885cc1f: Verifying Checksum
cb416885cc1f: Download complete
07e8f079852f: Verifying Checksum
07e8f079852f: Download complete
aad4571eacc0: Download complete
182039f5bee3: Verifying Checksum
182039f5bee3: Download complete
b3dd4e5cd318: Verifying Checksum
b3dd4e5cd318

## The following needs to be run in the terminal

In [11]:
!docker run -d --name adminer -p 8080:8080 adminer


3fe4a0b26d905ea26dbcdc30eb475f93feecdaa5332ea97315d7e2a157379ec7


Unable to find image 'adminer:latest' locally
latest: Pulling from library/adminer
f18232174bc9: Pulling fs layer
b1220d176a8d: Pulling fs layer
115c7a04f6c0: Pulling fs layer
0d55cff53123: Pulling fs layer
88d591373a19: Pulling fs layer
dc6d32971fca: Pulling fs layer
ffeef6b41a56: Pulling fs layer
6d95abe48138: Pulling fs layer
a80d3af78a9d: Pulling fs layer
f3fe38446507: Pulling fs layer
c6d0aade0806: Pulling fs layer
4f4fb700ef54: Pulling fs layer
e434f3dae19e: Pulling fs layer
8b847f2809a2: Pulling fs layer
93b26376816c: Pulling fs layer
52e9abf80869: Pulling fs layer
88d591373a19: Waiting
6d95abe48138: Waiting
a80d3af78a9d: Waiting
f3fe38446507: Waiting
dc6d32971fca: Waiting
c6d0aade0806: Waiting
ffeef6b41a56: Waiting
e434f3dae19e: Waiting
4f4fb700ef54: Waiting
8b847f2809a2: Waiting
93b26376816c: Waiting
52e9abf80869: Waiting
0d55cff53123: Waiting
115c7a04f6c0: Verifying Checksum
115c7a04f6c0: Download complete
f18232174bc9: Verifying Checksum
f18232174bc9: Download complete
0d55c

Run the following each in there own terminal

``` bash
docker logs -f mysql
docker logs -f adminer
```

In [21]:
!docker inspect mysql | findstr -n1 "IPAddress\":"
# or
print("-"*30)
show_container_ip("mysql")


213:            "IPAddress": "172.17.0.2",
227:                    "IPAddress": "172.17.0.2",
------------------------------
mysql in network 'bridge' has IP: 172.17.0.2


FINDSTR: /1 ignored


In [22]:
!docker inspect adminer| findstr  -n1 "IPAddress\":"
# or
print("-"*30)
show_container_ip("adminer")


210:            "IPAddress": "",
224:                    "IPAddress": "",
------------------------------
adminer in network 'bridge' has IP: 


FINDSTR: /1 ignored


### Lets do a bit of cleanup 

``` bash
#lets remove our old container
!docker stop adminer
!docker rm adminer
!docker stop mysql
!docker rm mysql
!docker stop nginx 
!docker rm nginx
```

In [ ]:
def clean_containers():
    containers = client.containers.list(all=True)
    if not containers:
        print("No containers to clean.")
        return
    
    for container in containers:
        container.stop()
        print(f"Waiting for {container.name} to stop...")
        while container.status != 'exited':
            pass
        
        container.reload()
        if container.status == 'exited':
            print(f"Removing {container.name}")
            container.remove()

list_running_containers()

clean_containers()


## Setting up the network

In [14]:
# Create a network called yourNetwork using the following command

!docker network create theHive

02fdcf2c6a7c21401fee3072bc71f5a5c93fd81bf6693fb108e445f911b52e46


In [15]:
!docker network inspect theHive

[
    {
        "Name": "theHive",
        "Id": "02fdcf2c6a7c21401fee3072bc71f5a5c93fd81bf6693fb108e445f911b52e46",
        "Created": "2025-04-28T02:01:26.035500851Z",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": {},
            "Config": [
                {
                    "Subnet": "172.18.0.0/16",
                    "Gateway": "172.18.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {},
        "Options": {},
        "Labels": {}
    }
]


In [36]:
!docker run -d --network theHive --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8


docker: Error response from daemon: Conflict. The container name "/mysql" is already in use by container "5e66c302f1d2f7e672327ffe8b914418b801b3e308ab4141e2aa0a44f8b61de3". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


In [17]:
!docker run -d --network theHive --name adminer -p 8080:8080 adminer

docker: Error response from daemon: Conflict. The container name "/adminer" is already in use by container "3fe4a0b26d905ea26dbcdc30eb475f93feecdaa5332ea97315d7e2a157379ec7". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


In [33]:
#Now lets connect the adminer
!docker network connect yourNetwork adminer

In [34]:
!docker inspect mysql |findstr -C2 " \"Networks\""
!docker inspect mysql |findstr -A9 " \"Networks\""| findstr " \"IPAddress\""
#or
print("-"*30)
show_container_ip("mysql")

            "Networks": {


FINDSTR: /C ignored
FINDSTR: /2 ignored
FINDSTR: /A ignored

------------------------------
mysql in network 'bridge' has IP: 172.17.0.2



FINDSTR: /9 ignored


In [35]:
!docker inspect adminer| findstr -A2 " \"Networks\""
!docker inspect adminer| findstr -A9 " \"Networks\""| findstr " \"IPAddress\""
!ping -c 1 172.18.0.3
#or
print("-"*30)
show_container_ip("adminer")

            "Networks": {


FINDSTR: /A ignored
FINDSTR: /2 ignored
FINDSTR: /A ignored
FINDSTR: /9 ignored


Access denied. Option -c requires administrative privileges.
------------------------------
adminer in network 'bridge' has IP: 
adminer in network 'yourNetwork' has IP: 


# LAB 
As a reminder you should be submitting to me 2 files. 
- _YourName_Lab 09.ipynb
    - this is your lab report
- _YourName__\<name of your script\>.py

## Part 1 - warm up

Now that you have 2 containers up and running and know there connected. Create a python script that will check to see if the IP addresses of the containers are up and running. 



In [11]:
import os

ips = ["172.17.0.2", "172.17.0.3"]

for ip in ips:
    response = os.system(f"ping -n 1 {ip} >nul")
    if response == 0:
        print(f"{ip} is ONNNNNNNN")
    else:
        print(f"{ip} is DOOOOOWWWNNNN")

172.17.0.2 is DOOOOOWWWNNNN
172.17.0.3 is DOOOOOWWWNNNN



## Part 2
Continue with the above code and create a script that will:
- Relaunch a container if it is ever shutdown. 
- Perform a maintenance restart after 1 day has passed. 
- Monitor the log files and outputs errors and warnings.

your python file does not need to be in a docker container


https://docs.docker.com/engine/api/sdk/examples/



In [12]:
import docker
import time
from datetime import datetime, timedelta

client = docker.from_env()

containers_to_watch = ["mysql", "nginx"]
start_time = datetime.now()

while True:
    for name in containers_to_watch:
        try:
            container = client.containers.get(name)
            if container.status != "running":
                print(f"{name} was stopped. Restarting...")
                container.restart()
        except docker.errors.NotFound:
            print(f"{name} not found.")
    
    now = datetime.now()
    if now - start_time >= timedelta(days=1):
        print("One day passed. Performing maintenance restart.")
        for name in containers_to_watch:
            try:
                container = client.containers.get(name)
                container.restart()
            except docker.errors.NotFound:
                print(f"{name} not found during maintenance restart.")
        start_time = datetime.now()

    for name in containers_to_watch:
        try:
            container = client.containers.get(name)
            logs = container.logs(tail=10).decode().lower()
            if "error" in logs or "warning" in logs:
                print(f"Log issues found in {name}:")
                print(logs)
        except docker.errors.NotFound:
            pass

    time.sleep(10)


Log issues found in mysql:
'/var/lib/mysql/mysql.sock' -> '/var/run/mysqld/mysqld.sock'
2025-04-28t02:47:02.416009z 0 [system] [my-015015] [server] mysql server - start.
2025-04-28t02:47:02.658697z 0 [system] [my-010116] [server] /usr/sbin/mysqld (mysqld 8.4.5) starting as process 1
2025-04-28t02:47:02.668997z 1 [system] [my-013576] [innodb] innodb initialization has started.
2025-04-28t02:47:02.949021z 1 [system] [my-013577] [innodb] innodb initialization has ended.
2025-04-28t02:47:03.211326z 0 [warning] [my-010068] [server] ca certificate ca.pem is self signed.
2025-04-28t02:47:03.211373z 0 [system] [my-013602] [server] channel mysql_main configured to support tls. encrypted connections are now supported for this channel.
2025-04-28t02:47:03.216274z 0 [warning] [my-011810] [server] insecure configuration for --pid-file: location '/var/run/mysqld' in the path is accessible to all os users. consider choosing a different directory.
2025-04-28t02:47:03.259733z 0 [system] [my-011323] [se

KeyboardInterrupt: 